<a href="https://colab.research.google.com/github/GiliardGodoi/tj-datasets/blob/main/notebooks/preprocessamento/2024_04_17_Extra%C3%A7%C3%A3o_Fragmentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, userdata
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install "tjdatasets @ git+https://github.com/GiliardGodoi/tj-datasets@april-24"

  Cloning https://github.com/GiliardGodoi/tj-datasets (to revision april-24) to /tmp/pip-install-2i0vqfkx/tjdatasets_4fe5c7379bff4e9c848bb3e96b208ded
  Running command git clone --filter=blob:none --quiet https://github.com/GiliardGodoi/tj-datasets /tmp/pip-install-2i0vqfkx/tjdatasets_4fe5c7379bff4e9c848bb3e96b208ded
  Running command git checkout -b april-24 --track origin/april-24
  Switched to a new branch 'april-24'
  Branch 'april-24' set up to track remote branch 'april-24' from 'origin'.
  Resolved https://github.com/GiliardGodoi/tj-datasets to commit eede31ef99c60a46d1ff7a586f9ef436fefe4320
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tjdatasets: filename=tjdatasets-0.0.3-py3-none-any.whl size=18453 sha256=b6fca4b48184aca4d544738bfa27d78947ebe610f08671732bac9f091d88bb1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-70yotng2

In [ ]:
import pandas as pd

from pathlib import Path

In [ ]:
from tjdatasets.fragmentador import processa_acordao

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from tjdatasets.utils import (PATTERN_REMOVE_SPECIAL_CHARS,
                              PATTERN_REMOVE_EXTRA_SPACE)

In [ ]:
BASE_FOLDER = Path('/content/drive/Shareddrives/Projeto_TJSP/Projeto Análise de Precedentes/Datasets/base_set_2023/RAW')
OUTPUT_FOLDER = Path('/content/drive/Shareddrives/Projeto_TJSP_Datasets/Datasets/base_abril_2024/PREPROCESSED')

base_33k_filepath = BASE_FOLDER / "acordaos_principais_33k.csv"
base_40k_filepath = BASE_FOLDER / "acordaos_principais_40k.csv"

assert base_33k_filepath.exists()
assert base_40k_filepath.exists()
assert OUTPUT_FOLDER.exists()

In [ ]:
for filepath in [base_33k_filepath, base_40k_filepath]:
    print('Processando...', filepath.name)

    df = pd.read_csv(filepath, encoding='utf8')
    print('Lido...', df.shape)
    df['formatado'] = (df['conteudo']
                   .str.replace(PATTERN_REMOVE_SPECIAL_CHARS, ' ', regex=True)
                   .str.replace(PATTERN_REMOVE_EXTRA_SPACE, ' ', regex=True)
                   .str.lower()
                   .str.translate( str.maketrans('áàãâäéèêëóòõôöíìîïúùüç', 'aaaaaeeeeoooooiiiiuuuc'))
                   )

    frame = (df.apply(processa_acordao, text_column='formatado', axis=1)
                    .apply(pd.Series))

    dest = OUTPUT_FOLDER / f'{filepath.stem}_Fragmentos.parquet.gzip'
    print('Salvando...', dest)
    frame.to_parquet(dest, compression='gzip', index=False)

Processando... acordaos_principais_33k.csv
Lido... (33016, 5)
Salvando... /content/drive/Shareddrives/Projeto_TJSP_Datasets/Datasets/base_abril_2024/PREPROCESSED/acordaos_principais_33k_Fragmentos.parquet.gzip
Processando... acordaos_principais_40k.csv
Lido... (41374, 5)
Salvando... /content/drive/Shareddrives/Projeto_TJSP_Datasets/Datasets/base_abril_2024/PREPROCESSED/acordaos_principais_40k_Fragmentos.parquet.gzip


In [ ]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41374 entries, 0 to 41373
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   nome_relator           41374 non-null  object
 1   nome_relator_regra     41374 non-null  int64 
 2   decisao_acordao        41374 non-null  object
 3   decisao_acordao_regra  41374 non-null  int64 
 4   decisao_unanime        41374 non-null  bool  
 5   voto_relator           41374 non-null  object
 6   voto_relator_regra     41374 non-null  int64 
 7   relatorio              33180 non-null  object
 8   relatorio_regra        41374 non-null  int64 
 9   fundamentacao          33180 non-null  object
 10  fundamentacao_regra    41374 non-null  int64 
 11  voto_divergente        2057 non-null   object
 12  voto_divergente_regra  41374 non-null  int64 
dtypes: bool(1), int64(6), object(6)
memory usage: 3.8+ MB
